In [ ]:
import os
import numpy as np
import pandas as pd

np.random.seed(125)

from keras.models import load_model
from sklearn.externals import joblib

output_folder = './output'
model_filename = 'final_model.hdf5'

keras_model = load_model(os.path.join(output_folder, model_filename))
print(keras_model.summary())

vectorizer_name = 'vectorizer'
vectorizer = joblib.load(os.path.join(output_folder, vectorizer_name))
print('{} loaded!'.format(vectorizer_name))

In [ ]:
import onnxmltools

deployment_folder = 'deploy'
onnx_export_folder = 'onnx'

# Convert the Keras model to ONNX
onnx_model_name = 'reviews_classifier.onnx'
converted_model = onnxmltools.convert_keras(keras_model, onnx_model_name, target_opset=7)

# Save the model locally...
onnx_model_path = os.path.join(deployment_folder, onnx_export_folder)
os.makedirs(onnx_model_path, exist_ok=True)
onnxmltools.utils.save_model(converted_model, os.path.join(onnx_model_path,onnx_model_name))

In [ ]:
import onnxruntime
# Load the ONNX model and observe the expected input shape
onnx_session = onnxruntime.InferenceSession(
    os.path.join(os.path.join(deployment_folder, onnx_export_folder), onnx_model_name))
input_name = onnx_session.get_inputs()[0].name
output_name = onnx_session.get_outputs()[0].name
print('Expected input shape: ', onnx_session.get_inputs()[0].shape)

In [ ]:
import nltk
nltk.download('stopwords')

In [ ]:
import sys
data_location = './data'
sys.path.append(data_location)
import textanalytics as ta

In [ ]:
nltk.download('punkt')

In [ ]:
test_review = ['The room was very nice and the beds were especially comfortable.']
test_review = ta.normalize_corpus(test_review)
test_review = vectorizer.transform(test_review)

test_review = test_review.toarray().astype(np.float32)
print(test_review.shape)

In [ ]:
# Run an ONNX session to classify the sample.
print('ONNX prediction: ', onnx_session.run([output_name], {input_name : test_review}))

# Use Keras to make predictions on the same sample
print('Keras prediction: ', keras_model.predict(test_review))

In [ ]:
# Next we will compare the performance of ONNX vs Keras
import timeit
n = 20000

In [ ]:
start_time = timeit.default_timer()
for i in range(n):
    keras_model.predict(test_review)
keras_elapsed = timeit.default_timer() - start_time
print('Keras performance: ', keras_elapsed)

In [ ]:
start_time = timeit.default_timer()
for i in range(n):
    onnx_session.run([output_name], {input_name : test_review})
onnx_elapsed = timeit.default_timer() - start_time
print('ONNX performance: ', onnx_elapsed)
print('ONNX is about {} times faster than Keras'.format(round(keras_elapsed/onnx_elapsed)))

In [ ]:
!cat .azureml/config.json

In [ ]:
import azureml.core

print(azureml.core.VERSION)

from azureml.core.workspace import Workspace

ws = Workspace.from_config()
print(ws)

In [ ]:
#Register the model and vectorizer
from azureml.core.model import Model

registered_model_name = 'reviews_classifier_onnx'
onnx_model_path = os.path.join(os.path.join(deployment_folder, onnx_export_folder), onnx_model_name)

registered_model = Model.register(model_path = onnx_model_path, # this points to a local file
                       model_name = registered_model_name, # this is the name the model is registered with         
                       description = "Reviews classification model.",
                       workspace = ws)

print(registered_model.name, registered_model.description, registered_model.version)

output_folder = './output'
vectorizer_name = 'vectorizer'
vectorizer_path = os.path.join(output_folder, vectorizer_name)

registered_vectorizer = Model.register(model_path = vectorizer_path, # this points to a local file
                       model_name = vectorizer_name, # this is the name the model is registered with         
                       description = "Reviews classification model vectorizer.",
                       workspace = ws)

print(registered_vectorizer.name, registered_vectorizer.description, registered_vectorizer.version)

In [ ]:
cwd = os.getcwd()
if cwd.endswith(deployment_folder):
    os.chdir('../')

In [ ]:
%%writefile $deployment_folder/scoring_service.py
import json
import numpy as np
import os
import sys
import urllib.request
import nltk
from sklearn.externals import joblib
from azureml.core.model import Model
import onnxruntime

onnx_model_name = 'reviews_classifier_onnx'
vectorizer_name = 'vectorizer'

def init():

    global onnx_session
    global vectorizer
    
    try:
        # Takes at most a couple of minutes to download all NLTK content
        print("downloading nltk.")
        nltk.download("all")
        
        tempFolderName = './resources'
        os.makedirs(tempFolderName, exist_ok=True)
        print('Content files will be saved to {0}'.format(tempFolderName))
        
        base_data_url = 'https://raw.githubusercontent.com/idiWork/Experiment_102/master/'
        filesToDownload = ['contractions.py', 'textanalytics.py']
        
        for file in filesToDownload:
            data_url = os.path.join(base_data_url, file)
            local_file_path = os.path.join(tempFolderName, file)
            urllib.request.urlretrieve(data_url, local_file_path)
            print('Downloaded file: ', file)
        
        print('Importing textanalytics...')
        sys.path.append(tempFolderName)
        import textanalytics as ta
        print('Done importing textanalytics.')
        
        # Retrieve the path to the model file using the model name
        onnx_model_path = Model.get_model_path(onnx_model_name)
        print('onnx_model_path: ', onnx_model_path)
        
        vectorizer_path = Model.get_model_path(vectorizer_name)
        print('vectorizer_path: ', vectorizer_path)
        
        onnx_session = onnxruntime.InferenceSession(onnx_model_path)
        print('Onnx Inference Session Created!')
        
        vectorizer = joblib.load(vectorizer_path)
        print('Vectorizer Loaded!')
    except Exception as e:
        print(e)

def run(raw_data):
    try:
        print("Received input: ", raw_data)
        
        print('Importing textanalytics...')
        import textanalytics as ta
        print('Done importing textanalytics.')
        
        print('Processing input...')
        input_data = np.array(json.loads(raw_data))
        input_data = ta.normalize_corpus(input_data)
        input_data = vectorizer.transform(input_data)
        input_data = input_data.toarray().astype(np.float32)
        print('Done processing input.')
        
        # Run an ONNX session to classify the input.
        result = onnx_session.run(None, {onnx_session.get_inputs()[0].name: input_data})[0].argmax(axis=1).item()
        # return just the classification index (0 or 1)
        return result
    except Exception as e:
        print(e)
        error = str(e)
        return error

In [ ]:
# create a Conda dependencies environment file
print("Creating conda dependencies file locally...")
from azureml.core.conda_dependencies import CondaDependencies 
conda_packages = ['numpy', 'scikit-learn']
pip_packages = ['nltk', 'azureml-sdk', 'onnxruntime']
mycondaenv = CondaDependencies.create(conda_packages=conda_packages, pip_packages=pip_packages)

cwd = os.getcwd()
if not cwd.endswith(deployment_folder):
    os.chdir(deployment_folder)
    
conda_file = 'dependencies.yml'
with open(conda_file, 'w') as f:
    f.write(mycondaenv.serialize_to_string())

runtime = 'python'
execution_script = 'scoring_service.py'

# create container image configuration
print("Creating container image configuration...")
from azureml.core.image import ContainerImage
image_config = ContainerImage.image_configuration(execution_script = execution_script, 
                                                  runtime = runtime, conda_file = conda_file)

# create the image
image_name = 'review-classifier-image'

from azureml.core import Image
image = Image.create(name=image_name, models=[registered_model, registered_vectorizer], 
                     image_config=image_config, workspace=ws)

# wait for image creation to finish
image.wait_for_creation(show_output=True)

os.chdir("..")

In [ ]:
from azureml.core.webservice import AciWebservice, Webservice

aci_config = AciWebservice.deploy_configuration(
    cpu_cores = 1, 
    memory_gb = 1, 
    tags = {'name': 'Review Classification'}, 
    description = "Classifies a review as room (0), diner (1) or pool (2).")

service_name = "reviewclassservice"

aci_service = Webservice.deploy_from_image(deployment_config=aci_config, 
                                           image=image, 
                                           name=service_name, 
                                           workspace=ws)

aci_service.wait_for_deployment(show_output=True)

In [ ]:
import json

test_reviews = ['The room was very nice and the beds were especially comfortable.', 
               'The kids loved going to the Kids Club at the swimming pool.', 
               'The food was great and the buffet was priced very reasonably.']

for i in range(len(test_reviews)):
    result = aci_service.run(json.dumps([test_reviews[i]]))
    print('Predicted label for test review #{} is {}'.format(i+1, result))

In [ ]:
import requests

url = aci_service.scoring_uri
print('ACI Service: Review Classification scoring URI is: {}'.format(url))
headers = {'Content-Type':'application/json'}

for i in range(len(test_reviews)):
    response = requests.post(url, json.dumps([test_reviews[i]]), headers=headers)
    print('Predicted label for test review #{} is {}'.format(i+1, response.text))